주소 컬럼의 빈값 채우기, 해시태그 컬럼의 공백 및 특수기호 등 제거 등 전처리   

In [ ]:
import json, folium
import requests

In [ ]:
#  데이터 컬럼명 변경하기
df.rename(columns ={'0':'관광지명','1':'주소','2':'연락처','3':'상세정보','4':'해시태그','5':'상세페이지_url'}, inplace=True)

In [ ]:
# 주소 컬럼이 null값인 행 확인하기
df[df['주소'].isnull()]

In [ ]:
# 주소가 빈값인 관광지명 리스트로 만들기
search_list = df[df['주소'].isnull()]['관광지명'].values.tolist()

In [ ]:
with open('도로명주소APIkey.txt') as file:
    road_key = file.read()

In [ ]:
base_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f'keyword={quote ("렛츠런팜")}&resultType=json'
url = f'{base_url}?{params1}&{params2}'

In [ ]:
result = requests.get(url)
result.status_code

In [ ]:
json.loads(result.text)['results']['juso'][0]['roadAddr']

In [ ]:
#  null값인 주소 변경하기
df.loc[849,'주소'] = '제주특별자치도 제주시 조천읍 남조로 1649-48'
df.loc[883,'주소'] = '제주특별자치도 제주시 구좌읍 구좌해안로 237' 
df.loc[955,'주소'] = '제주 제주시 우도면 연평리 3'
df.loc[967,'주소'] = '제주특별자치도 서귀포시 안덕면 사계리 112-3'
df.loc[982,'주소'] = '제주 제주시 애월읍 평화로 1529'
df.loc[999,'주소'] = '제주 서귀포시 안덕면 상천리 산24'
df.loc[1014,'주소'] = '제주특별자치도 제주시 일도일동 1498'
df.loc[1073,'주소'] = '제주특별자치도 서귀포시 남원읍 신흥리 2083'
df.loc[1176,'주소'] = '제주특별자치도 서귀포시 안덕면 감산리 1803'
df.loc[1342,'주소'] = '제주특별자치도 제주시 추자면 영흥리'
df.loc[1362,'주소'] = '제주특별자치도 서귀포시 서귀동'

In [ ]:
#  해시태그_sub 열 생성 - 해시태그 열의 데이터값 공백 및 특수기호 등 제외
df['해시태그_sub'] = df['해시태그'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)

주소의 위도 경도 가져오기

In [ ]:
with open('카카오APIkey.txt') as file:
    road_key = file.read()

In [ ]:
lat_list, lng_list = [],[]
for i in df.index:
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    try:
        lat_list.append(result['documents'][0]['y'])
        lng_list.append(result['documents'][0]['x'])
    except:
        print(df.관광지명[i])

In [ ]:
df['위도'] = lat_list
df['경도'] = lng_list

In [ ]:
#  데이터 저장하기
df.to_csv('./관광지정보최종.csv', index = False)
pd.read_csv('./관광지정보최종.csv')
df=pd.read_csv('./관광지정보최종.csv')

지도 그리기

In [ ]:
import folium
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family']='Malgun Gothic'
plt.rcParams['axes.unicode_minus']= False

In [ ]:
# 지도 표시하기
map = folium.Map(location = [df.위도.mean(), df.경도.mean()], zoomstart = 14)
for i in df.index:
    folium.Marker(
        location = [df['위도'][i], df['경도'][i]], 
        tooltip = df.관광지명[i],
        icon=folium.Icon( color='pink', icon= 'star'),
        popup = folium.Popup(df.주소[i],df.해시태그[i], max_width=200) 
    ).add_to(map)

In [ ]:
# 지도 표시하기__제주시와 그 외(서귀포시)로 나뉘어서 표시하기
map = folium.Map(location = [df.위도.mean(), df.경도.mean()], zoomstart = 14)
for i in df.index:
    if '제주시' in df['주소'][i]:
        folium.Marker(
            location = [df['위도'][i], df['경도'][i]], 
            tooltip = dft.관광지명[i],
            icon=folium.Icon( color='beige', icon= 'cloud'),
            popup = folium.Popup(df.주소[i], df.해시태그[i], max_width=200) ).add_to(map)
    else:
        folium.Marker(
            location = [df['위도'][i], df['경도'][i]], 
            tooltip = df.관광지명[i],
            icon=folium.Icon( color='lightgreen', icon= 'glyphicon-star'),
            popup = folium.Popup(df.주소[i], df.해시태그[i], max_width=200) ).add_to(map)